## 참고 :  https://velog.io/@vector13/Food101-데이터셋을-이용한-음식-이미지-분류기-만들기

### 1. 기본 환경 설정 

In [1]:
from shutil import copy
from collections import defaultdict
import scipy


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import SGD

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

2.10.0
/device:GPU:0
Metal device set to: Apple M1


### 2. food101 다운 소스 : https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/
* 주의 : food101 데이터 다운받은 후 train/test 나눌 때 한번만 코드 실행

In [12]:
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

# 본인 환경에 맞게 food101 들어있는 디렉토리 설정하기
prepare_data('/Volumes/T7/food-101/meta/train.txt', '/Volumes/T7/food-101/images', '/Volumes/T7/food-101/train')
prepare_data('/Volumes/T7/food-101/meta/test.txt', '/Volumes/T7/food-101/images', '/Volumes/T7/food-101/test')
   


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

### 3. food101 데이터 기본 세팅 (전처리)

In [2]:
# releases the global state: avoid clutter from old models and layers
K.clear_session()

In [3]:
# 초기 변수 설정
n_classes = 101
img_width, img_height = 299, 299
train_data_dir = '/Volumes/T7/food-101/train'
validation_data_dir = '/Volumes/T7/food-101/test'
nb_train_samples = 75750       # 750 * 101
nb_validation_samples = 25250  # 250 * 101
batch_size = 20                # 한 번 실행될 때 생성할 이미지 수

In [4]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,       # multiply the data by the value
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True)   # Randomly flip inputs horizontally

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [5]:
# flow_from_directory : meta data가 담긴 디렉토리 설정
# meta data : 이미지 데이터 파일들과 해당 이미지들이 무슨 이미지를 나타내는지 텍스트로 표현한 문자열
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


### 4. 학습 모델 생성

In [6]:
# MobileNetV2 모델 생성

mbv2 = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (299,299,3))
# weights : None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
# include_top : include the fully-connected layer at the top of the network
x = mbv2.output
x = GlobalAveragePooling2D()(x)     # pooling layer
x = Dense(128,activation='relu')(x) # hidden layer : model can learn more complex functions and classify for better results.
x = Dropout(0.2)(x)                 # 과적합을 방지하기 위해 무작위로 특정 노드(입력값)를 0으로 만든다. (여기서는 20%)
predictions = Dense(101,kernel_regularizer = regularizers.l2(0.005), activation = 'softmax')(x) #  #final layer : with softmax activation for 101 classes

model = Model(inputs = mbv2.input, outputs = predictions)
model.compile(optimizer=SGD(learning_rate = 0.0001, momentum = 0.9), loss = 'categorical_crossentropy', metrics = ['accuracy'])

### 5. 모델 학습시키기

In [ ]:
checkpointer = ModelCheckpoint(filepath='best_model_3class_sept.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history.log')

# 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용
history = model.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,      #  한 epoch에 사용한 스텝 수
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  # 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수
                    epochs=10,                                             # 전체 훈련 데이터셋에 대해 학습 반복 횟수
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained.h5')

/var/folders/cc/6xhsqlks6mg1kzb03g_1zn7r0000gn/T/ipykernel_14141/911124647.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
 787/3787 [=====>........................] - ETA: 25:30 - loss: 5.1840 - accuracy: 0.0163

### 6. 모델 예측하기 (predict_generator)

### 7. 모델 사용하기 (predict_generator)

In [ ]:
from keras import utils
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.models import load_model

In [ ]:
#creating a list of all the foods, in the argument i put the path to the folder that has all folders for food
def create_foodlist(path):
    list_ = list()
    for root, dirs, files in os.walk(path, topdown=False):
      for name in dirs:
        list_.append(name)
    return list_    

#loading the model i trained and finetuned        
my_model = load_model('model_trained.h5', compile = False)
food_list = create_foodlist("food101/images")

#function to help in predicting classes of new images loaded from my computer(for now) 
def predict_class(model, images, show = True):
  for img in images:
    img = utils.load_img(img, grayscale=False, color_mode='rgb', target_size=(299, 299))

    img = utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)         
    img /= 255.                                      

    pred = model.predict(img)
    index = np.argmax(pred)    #Returns the indices of the maximum values along an axis, In case of multiple occurrences of the maximum values, the indices corresponding to the first occurrence are returned.
    food_list.sort()
    pred_value = food_list[index]
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred_value)
        plt.show()

#add the images you want to predict into a list (these are in the WD)
images = []
images.append('sc.jpg')
images.append('vb.jpg')
images.append('vp.jpg')
images.append('es.jpg')


print("PREDICTIONS BASED ON PICTURES UPLOADED")
predict_class(my_model, images, True)